In [45]:
import pandas as pd
import numpy as np
import json
import requests
from sqlalchemy import create_engine


### Extract CSVs into DataFrames

Incident & Shootings & Holidays -----------------------------------------------------------------------------------------------------------------------------------------

In [46]:
#Table of Incidents of gun violence in the USA
Incident = "all_incidents.csv"
Incident_df = pd.read_csv(Incident)
Incident2_df = Incident_df.rename(columns={"incident_id":"Incident ID",	"date":"Incident Date",	"state":"State","city":"City or County","address":"Address","n_killed":"# Killed","n_injured":"# Injured"})
Incident2_df['Incident Date'] = pd.to_datetime(Incident2_df['Incident Date'])
Incident2_df.head()


,Incident ID,Incident Date,State,City or County,Address,# Killed,# Injured
0,2314858,2022-05-28,Arkansas,Little Rock,W 9th St and Broadway St,0,1
1,2314409,2022-05-28,Colorado,Denver,3300 block of Clay St,0,1
2,2314498,2022-05-28,Missouri,Saint Louis,Page Blvd and Vandeventer Ave,0,1
3,2314485,2022-05-28,South Carolina,Florence,Old River Rd,0,2
4,2314483,2022-05-28,California,Carmichael,4400 block of Manzanita Ave,1,0


In [47]:
#Incidents of mass shootings in the USA
Mass_shooting = "mass_shootings.csv"
Massshooting_df = pd.read_csv(Mass_shooting)
Massshooting_df['Incident Date'] = pd.to_datetime(Massshooting_df['Incident Date'])

Massshooting_df.head()

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured
0,271363,2014-12-29,Louisiana,New Orleans,Poydras and Bolivar,0,4
1,269679,2014-12-27,California,Los Angeles,8800 block of South Figueroa Street,1,3
2,270036,2014-12-27,California,Sacramento,4000 block of May Street,0,4
3,269167,2014-12-26,Illinois,East St. Louis,2500 block of Summit Avenue,1,3
4,268598,2014-12-24,Missouri,Saint Louis,18th and Pine,1,3


In [48]:
US_Holiday_Dates = "US Holiday Dates (2004-2021).csv"
Holiday_df = pd.read_csv(US_Holiday_Dates)
Holiday_df['Date'] = pd.to_datetime(Holiday_df['Date'])
Holiday_df = Holiday_df.drop(columns=['Month','Day','Year'])
Holiday_df.head()




,Date,Holiday,WeekDay
0,2004-07-04,4th of July,Sunday
1,2005-07-04,4th of July,Monday
2,2006-07-04,4th of July,Tuesday
3,2007-07-04,4th of July,Wednesday
4,2008-07-04,4th of July,Friday


Population 2019, 2020, 2021 -----------------------------------------------------------------------------------------------------------------------------------------------------

In [49]:
Population_2019 = "2019_census_population.csv"
Pop2019_df = pd.read_csv(Population_2019)
Pop2019_df.head()

,STATE,POPESTIMATE2019,lat,long
0,Alabama,4903185,32.377716,-86.300568
1,Alaska,731545,58.301598,-134.420212
2,Arizona,7278717,33.448143,-112.096962
3,Arkansas,3017804,34.746613,-92.288986
4,California,39512223,38.576668,-121.493629


In [50]:
def location(lat, long):
     url = 'http://maps.googleapis.com/maps/api/geocode/json?latlng={0},{1}&sensor=false'.format(lat, long)
     r = requests.get(url)
     r_json = r.json()
     if len(r_json['results']) < 1: return None, None
     res = r_json['results'][0]['address_components']
     country  = next((c['long_name'] for c in res if 'country' in c['types']), None)
     locality = next((c['long_name'] for c in res if 'locality' in c['types']), None)
     return locality, country

Pop2019_df['locality'] = np.vectorize(location)(Pop2019_df['lat'],Pop2019_df['long'])
Pop2019_df

ValueError: Length of values does not match length of index

In [36]:
Population_2020 = "2020_census_population.csv"
Pop2020_df = pd.read_csv(Population_2020)
Pop2020_df2 = Pop2020_df.rename(columns={'Population':'2020Population'})

Pop2020_df2.head()

,City,State,Type,Counties,2020Population,Latitude,Longitude
0,New York,NY,City,Bronx;Richmond;New York;Kings;Queens,8804190,40.714,-74.007
1,Los Angeles,CA,City,Los Angeles,3898747,34.052,-118.243
2,Chicago,IL,City,Cook;DuPage,2746388,41.882,-87.628
3,Houston,TX,City,Harris;Fort Bend;Montgomery,2304580,29.760,-95.363
4,Phoenix,AZ,City,Maricopa,1608139,33.448,-112.074


In [20]:
Population_2021 = "2021_estimate_population.csv"
Pop2021_df = pd.read_csv(Population_2021, sep=',', encoding='latin-1')
Pop2021_df.head()


,NAME,STNAME,POPESTIMATE2021
0,Alabama,Alabama,5039877
1,Abbeville city,Alabama,2379
2,Adamsville city,Alabama,4294
3,Addison town,Alabama,668
4,Akron town,Alabama,226
